In [6]:
import pandas as pd
import os

In [30]:
cwd = os.getcwd()

change_wd = os.chdir('C:/Users/rayli/Desktop/Algorithms/Project2/Algorithms_Project_2')
# print(cwd)
print(cwd)


C:\Users\rayli\Desktop\Algorithms\Project2\Algorithms_Project_2


In [29]:


# reading the CSV files
# some column names had empty spaces in the beginning of the name, "skipinitialspace=True" parameter removes empty spaces.
routes_data = pd.read_csv("C://Users/rayli/Desktop/Algorithms/Project2/Algorithms_Project_2/src/data/routes.csv", encoding='ISO-8859-1', skipinitialspace=True)

airlines_data = pd.read_csv("C:/Users/rayli/Desktop/Algorithms/Project2/Algorithms_Project_2/src/data/airlines.csv",encoding='ISO-8859-1',skipinitialspace=True)

# assigning column names for airports file because names are not provided in the original file
col_names_airports = [
    'Airport ID',
    'Name',
    'City',
    'Country',
    'IATA',
    'ICAO',
    'Latitude',
    'Longitude',
    'Altitude',
    'Timezone',
    'DST',
    'Tz',
    'Type',
    'Source'
]
airports_data = pd.read_csv('src/data/airports-extended.csv', encoding='ISO-8859-1', names=col_names_airports)

# assigning column names for planes file because names are not provided in the original file
col_names_planes = [
    "Plane",
    "IATA",
    "ICAO"
]
planes_data = pd.read_csv('src/data/planes.dat.txt', encoding='ISO-8859-1', names=col_names_planes)


# handling missing values
# replacing '\N' with NaN for consistency
airports_data.replace('\\N', pd.NA, inplace=True)
airlines_data.replace('\\N', pd.NA, inplace=True)
routes_data.replace('\\N', pd.NA, inplace=True)
planes_data.replace('\\N', pd.NA, inplace=True)


# displaying the contents of the CSV files
def print_routes_data():
    """
    COLUMN NAMES:
    "airline": 2-letter or 3-letter code of the airline.
    "airline ID": Unique OpenFlights identifier for airline.
    "source airport": 3-letter or 4-letter code of the source airport.
    "source airport id": Unique OpenFlights identifier for source airport.
    "destination airport": 3-letter or 4-letter code of the destination airport.
    "destination airport id": Unique OpenFlights identifier for destination airport.
    "codeshare": "Y" if this flight is a codeshare (that is, not operated by Airline, but another carrier), empty
    otherwise.
    "stops": Number of stops on this flight ("0" for direct).
    "equipment": 3-letter codes for plane type(s) generally used on this flight, separated by spaces.
    """
    print(routes_data.to_string())


def print_airlines_data():
    """
    COLUMN NAMES:
    "Airline ID": ID Unique OpenFlights identifier for this airline.
    "Name:" Name of the airline.
    "Alias": Alias of the airline. For example, All Nippon Airways is commonly known as "ANA".
    "IATA": 2-letter IATA code, if available.
    "ICAO": 3-letter ICAO code, if available.
    "Callsign": Airline callsign.
    "Country": Country or territory where airline is incorporated.
    "Active": "Y" if the airline is or has until recently been operational, "N" if it is defunct.
    """
    print(airlines_data.to_string())


def print_airports_data():
    """
    COLUMN NAMES:
    "Airport ID": Unique OpenFlights identifier for this airport.
    "Name": Name of airport. May or may not contain the City name.
    "City": Main city served by airport. May be spelled differently from Name.
    "Country": Country or territory where airport is located. See countries.dat to cross-reference to ISO 3166-1 codes.
    "IATA": 3-letter IATA code. Null if not assigned/unknown.
    "ICAO": 4-letter ICAO code. Null if not assigned.
    "Latitude": Decimal degrees, usually to six significant digits. Negative is South, positive is North.
    "Longitude": Decimal degrees, usually to six significant digits. Negative is West, positive is East.
    "Altitude": In feet.
    "Timezone": Hours offset from UTC. Fractional hours are expressed as decimals, eg. India is 5.5.
    "DST": Daylight savings time. One of E (Europe), A (US/Canada), S (South America), O (Australia), Z (New Zealand),
    N (None) or U (Unknown). See also: Help: Time
    "Tz": database time zone Timezone in "tz" (Olson) format, eg. "America/Los_Angeles".
    "Type": Type of the airport. Value "airport" for air terminals, "station" for train stations, "port" for ferry
    terminals and "unknown" if not known.
    "Source": Source of this data. "OurAirports" for data sourced from OurAirports, "Legacy" for old data not matched to
    OurAirports (mostly DAFIF), "User" for unverified user contributions. In airports.csv, only source=OurAirports
    is included.
    """
    print(airports_data.to_string())


def print_planes_data():
    """
    COLUMN NAMES:
    "Plane": plane model
    "IATA",
    "ICAO"
    """
    print(planes_data.to_string())


In [31]:
# To print the whole data set, use functions defined in data_import file. Ex: print_routes_data()

# To print individual columns, call by column name. Ex: print(routes_data[['airline']])

# Consider trips between destinations that involve no more than 1 layover.
less_layovers = routes_data[(routes_data.stops == 0)]

# Merging routs_data, airports_data, airlines_data and planes_data
# Convert desired columns to the same data type.
airports_data['Airport ID'] = airports_data['Airport ID'].astype(str)
less_layovers.loc[:, 'source airport id'] = less_layovers['source airport id'].astype(str)
less_layovers.loc[:, 'destination airport id'] = less_layovers['destination airport id'].astype(str)
airlines_data['Airline ID'] = airlines_data['Airline ID'].astype(str)
planes_data['IATA'] = planes_data['IATA'].astype(str)

# Merge to add the source city name.
merged_data = less_layovers.merge(
    airports_data[['Airport ID', 'City']],
    left_on='source airport id',
    right_on='Airport ID',
    how='left'
)
merged_data.rename(columns={'City': 'source city'}, inplace=True)
merged_data.drop('Airport ID', axis=1, inplace=True)  # removing the extra 'Airport ID' column.

# Merge to add the destination city name.
merged_data = merged_data.merge(
    airports_data[['Airport ID', 'City']],
    left_on='destination airport id',
    right_on='Airport ID',
    how='left'
)
merged_data.rename(columns={'City': 'destination city'}, inplace=True)
merged_data.drop('Airport ID', axis=1, inplace=True)  # removing the extra 'Airport ID' column again.

# Merge airlines data with routes to get the name of the airline.
merged_data = merged_data.merge(
    airlines_data[['Airline ID', 'Name']],
    left_on='airline ID',
    right_on='Airline ID',
    how='left'
)
merged_data.rename(columns={'Name': 'airline name'}, inplace=True)
merged_data.drop('Airline ID', axis=1, inplace=True)  # removing the extra 'Airline ID' column.

# Merge planes data with routes to get the plane model.
# Split the equipment string into a list of equipment codes.
merged_data['equipment'] = merged_data['equipment'].str.split()

# Explode merged_data so each equipment code has its own row.
merged_data = merged_data.explode('equipment')

merged_data = merged_data.merge(
    planes_data[['IATA', 'Plane']],
    left_on='equipment',
    right_on='IATA',
    how='left'
)
merged_data.rename(columns={'Plane': 'plane model'}, inplace=True)
merged_data.drop('IATA', axis=1, inplace=True)  # removing the extra 'IATA' column.

# Drop NaN values if 'equipment' was not mapped to a specific plane model.
merged_data = merged_data.dropna(subset=['plane model'])

# Consider a source city, say New York, and a target city, say San Francisco.
ny_sf_direct_flights = merged_data[(merged_data["source city"] == "New York") &
                                   (merged_data["destination city"] == "San Francisco")]

ny_from_flights = merged_data[(merged_data["source city"] == "New York")]
sf_to_flights = merged_data[(merged_data["destination city"] == "San Francisco")]

# Merge ny_from_flights and sf_to_flights on the condition that the destination city of the NY flight is the same as the source city of the SF flight.
ny_sf_indirect_flights = pd.merge(
    ny_from_flights, sf_to_flights,
    left_on='destination city',
    right_on='source city',
    suffixes=(' ny', ' sf')
)

print(ny_sf_indirect_flights.to_string())
print(ny_sf_direct_flights.to_string())

# print(ny_sf_indirect_flights.shape)  # (3117, 26)
# print(ny_sf_direct_flights.shape)  # (8, 13)

ny_models = ny_sf_indirect_flights['plane model ny'].unique()
sf_models = ny_sf_indirect_flights['plane model sf'].unique()
ny_sf_models = ny_sf_direct_flights['plane model'].unique()

models_set = set(list(ny_models) + list(sf_models) + list(ny_sf_models))
sorted_models = sorted(models_set)
print(sorted_models)


     airline ny airline ID ny source airport ny source airport id ny destination apirport ny destination airport id ny codeshare ny  stops ny equipment ny source city ny destination city ny                            airline name ny                        plane model ny airline sf airline ID sf source airport sf source airport id sf destination apirport sf destination airport id sf codeshare sf  stops sf equipment sf     source city sf destination city sf                            airline name sf                         plane model sf
0            4O         17885               JFK                 3797                     MEX                      1824          NaN         0          320       New York         Mexico City                  Interjet (ABC Aerolineas)                           Airbus A320         AM           321               MEX                 1824                     SFO                      3469          NaN         0          737        Mexico City       San Francisc

# Test Algo Max/Min

Creating the Max/min

In [32]:
import numpy as np

# Creating a test_matrix_graph representing a network with 20 nodes
test_matrix_graph = np.zeros((20, 20))

# Adding edges to the test_matrix_graph
test_matrix_graph[0, 1] = 1
test_matrix_graph[0, 2] = 1
test_matrix_graph[0, 3] = 1
test_matrix_graph[1, 2] = 1
test_matrix_graph[1, 4] = 1
test_matrix_graph[2, 3] = 1
test_matrix_graph[2, 4] = 1
test_matrix_graph[3, 4] = 1
test_matrix_graph[4, 5] = 1
test_matrix_graph[4, 6] = 1
test_matrix_graph[5, 6] = 1
test_matrix_graph[5, 7] = 1
test_matrix_graph[6, 7] = 1
test_matrix_graph[7, 8] = 1
test_matrix_graph[7, 9] = 1
test_matrix_graph[8, 9] = 1
test_matrix_graph[9, 10] = 1
test_matrix_graph[9, 11] = 1
test_matrix_graph[10, 11] = 1
test_matrix_graph[11, 12] = 1
test_matrix_graph[11, 13] = 1
test_matrix_graph[12, 13] = 1
test_matrix_graph[13, 14] = 1
test_matrix_graph[13, 15] = 1
test_matrix_graph[14, 15] = 1
test_matrix_graph[15, 16] = 1
test_matrix_graph[15, 17] = 1
test_matrix_graph[16, 17] = 1
test_matrix_graph[17, 18] = 1
test_matrix_graph[17, 19] = 1
test_matrix_graph[18, 19] = 1

print(test_matrix_graph)


[[0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.